In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [43]:
batch_sz = 32
block_sz = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
no_embd = 32
eval_itters = 200
torch.manual_seed(1024)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [44]:
with open('file.txt','r', encoding="utf-8") as f:
    text = f.read()

In [45]:
# finding all the unique char in the `text`
unique_chars = ''.join(sorted(set(text)))
vocab_sz = len(unique_chars)

In [46]:
# creating a maping dict from `str` to ìnt` and `int` to `str` for the dataset:
stoi = {chr: i for i, chr in enumerate(unique_chars)}
itos = {i: chr for chr, i in stoi.items()}

In [47]:
encode = lambda inp : [stoi[alpha] for alpha in inp]
decode = lambda inp : [itos[intr] for intr in inp]

In [78]:
# converting the encoded data into a tensor.
dataset = torch.tensor(encode(text), dtype=torch.long)

# now let's split the above dataset into train and dev 80% & 20% respect.
dataLen = len(dataset)
train_per = int(dataLen * 0.8)
dev_per = int(dataLen * 0.2)

train_data = dataset[:train_per]
dev_data = dataset[:dev_per]

In [79]:
# creating a stack batch where the stack size is: 'stack_sz' with batch len of 8.
def get_batch(dt):
    data = train_data if dt == 'train' else dev_data
    ix = torch.randint(len(data) - block_sz, (batch_sz, ))
    x = torch.stack([data[i : i + block_sz] for i in ix])
    y = torch.stack([data[i + 1 : i + 1 + block_sz] for i in ix])
    return x, y

In [81]:
@torch.no_grad()
def estimated_loss():
    out = {}
    model.eval()
    for split in ['train', 'eval']:
        losses = torch.zeros(eval_itters)
        for k in range(eval_itters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            loss[k], = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [82]:
# B, T, C = 4, 8, 32
# head_size = 16
# x = torch.randn((B, T, C))

# key = nn.Linear(C, head_size, bias=False)
# query = nn.Linear(C, head_size, bias=False)

# k = key(x)
# q = query(x)

# tril = torch.tril(torch.ones(T, T))

# wei = q @ k.transpose(-2, -1) * head_size**-0.5

# wei = wei.masked_fill(tril == 0, float('-inf'))
# wei = F.softmax(wei, dim=-1)

# value = nn.Linear(C, head_size, bias=False)
# v = value(x)

# out = wei @ v

In [83]:
class Head(nn.Module):
    """ head of self-attention """
    def __init__(self, head_sz):
        super().__init__()
        self.key = nn.Linear(no_embd, head_size, bias=False)
        self.query = nn.Linear(no_embd, head_size, bias=False)
        self.value = nn.Linear(no_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_sz, block_sz)))
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        out = wei @ v
        return out

In [84]:
class MutiHeadAttention(nn.Module):
    """Multi Head attention in parallel"""
    
    def __init__(self, num_heads, head_sz):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_sz) for _ in range(num_heads)])
    
    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)

In [85]:
class BigramLanguageMode(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_sz, no_embd)
        self.position_embedding_table = nn.Embedding(block_sz, no_embd)
        self.sa_heads = MutiHeadAttention(4, no_embd//4)
        self.lm_head = nn.Linear(no_embd, vocab_sz)
        
    
    def forward(self, idx, target=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = tok_emb + pos_emb
        x = self.sa_heads(x)
        
        logits = self.lm_head(x)
        
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target) 
            
        return logits, loss
    
    def generate(self, idx, max_new_token):
        
        for _ in range(max_new_token):
            idx_cond = logits[:, -block_sz:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_idx = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_idx), dim=1)

        return idx


In [90]:
model = BigramLanguageMode()

In [91]:
# traning the model.
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)

for iter in range(max_iters):
    
#     if iter % eval_interval == 0:
#         losses = estimated_loss()
#         print(f"step {iter}: train loss {losses['train']:.4f }, val loss {losses['val']:.4f }")
    
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    
    loss.backward()
    optimizer.step()
    
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_token=500)[0].tolist()))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x64 and 32x65)